In [0]:
import pandas as pd 
import numpy as np
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist  import input_data
%matplotlib inline

In [0]:
mnist = input_data.read_data_sets('datset/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting datset/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting datset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting datset/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting datset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/mo

In [0]:
# helper function to initialise weights of a layer 

def weights(shape):

  return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

In [0]:
# helper function to initialise bias of a layer 

def bias(shape):

  return tf.Variable(tf.constant(0.1,shape=shape))

In [0]:
# helper function to build convo2d

def conv2d(x,W):
  # x =  input tensor (batches,h,w,channels)
  # W = kernel(filter_h,filter_w,channels IN, channels out)

  return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [0]:
# helper function for pooling

def max_pooling(x):
#   ksize means we have a polling of (2*2) were we would look for maximum value in a 2*2 pool and we would move with a stride of 2*2
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [0]:
# creating layer Convolutional layer
def convolutional_layer(input_x,shape):

  # in CNN common diagram we have a 5*5 convolutional layer with 32 features in it .
  # ie the shape is [5,5,1,32] , this is the shape that will be passed in here in the SHAPE parameter

  W = weights(shape)
  print("convo layer W value ",W)
  print("convo layer shape value ",shape[3])
  b = bias([shape[3]])# the shape's 3rd element is the no of features (32)
  print("convo layer b value ",b)
  return tf.nn.relu(conv2d(input_x,W)+b)

In [0]:
# fully connected layer

def normal_full_layer(input_layer,size):
  input_size = int(input_layer.get_shape()[1])
  print("input size ", input_size)
  W = weights([input_size,size])
  b = bias([size])
  return tf.matmul(input_layer,W) + b

In [0]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [0]:
# layer 
# input /image layer

X_image = tf.reshape(x,[-1,28,28,1])


In [0]:
convo_1 = convolutional_layer(X_image,shape=[5,5,1,32]) # patch size (5,5) , grayscale [1], features [32]
convo_1_pooling = max_pooling(convo_1)

convo layer W value  <tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>
convo layer shape value  32
convo layer b value  <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>


In [0]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64]) # patch size (5,5) , grayscale [1], features [32]
convo_2_pooling = max_pooling(convo_2)

convo layer W value  <tf.Variable 'Variable_2:0' shape=(5, 5, 32, 64) dtype=float32_ref>
convo layer shape value  64
convo layer b value  <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>


In [0]:
convo_2_flat = tf.reshape(convo_2_pooling,shape=[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

input size  3136


In [0]:
# dropiut

hold_prob =  tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
y_pred = normal_full_layer(full_one_dropout,10)

input size  1024


In [0]:
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
optimiser = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy_loss)

In [0]:
saver = tf.train.Saver()

In [0]:
init = tf.global_variables_initializer()
steps = 5000
with tf.Session() as sess:
  sess.run(init)

  for i in range(steps):

    batch_x, batch_y = mnist.test.next_batch(50)

    sess.run(optimiser,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})

    if i %100 == 0:
      print("Accuracy on step :{}".format(i))

      corrected_pred = tf.equal(tf.argmax(y_pred,1),(tf.argmax(y_true,1)))
      accuracy = tf.reduce_mean(tf.cast(corrected_pred,tf.float32))
      print(sess.run(accuracy,feed_dict={x:mnist.train.images,y_true:mnist.train.labels,hold_prob:1.0}))
      print('\n')
  saver.save(sess,'model/cnn_mnist_tf.h5')


Accuracy on step :0
0.09821818


Accuracy on step :100
0.93585455


Accuracy on step :200
0.94914544


Accuracy on step :300
0.9684909


Accuracy on step :400
0.9690727


Accuracy on step :500
0.9684909


Accuracy on step :600
0.97498184


Accuracy on step :700
0.97156364


Accuracy on step :800
0.97136366


Accuracy on step :900
0.97505456


Accuracy on step :1000
0.9778909


Accuracy on step :1100
0.9772182


Accuracy on step :1200
0.97123635


Accuracy on step :1300
0.9767454


Accuracy on step :1400
0.977


Accuracy on step :1500
0.9773273


Accuracy on step :1600
0.9772


Accuracy on step :1700
0.9790546


Accuracy on step :1800
0.9748545


Accuracy on step :1900
0.9763273


Accuracy on step :2000
0.97876364


Accuracy on step :2100
0.97483635


Accuracy on step :2200
0.9760909


Accuracy on step :2300
0.9796364


Accuracy on step :2400
0.9769091


Accuracy on step :2500
0.97245455


Accuracy on step :2600
0.9777091


Accuracy on step :2700
0.9748727


Accuracy on step :2800
0.972